In [1]:
!nvidia-smi
!apt-get install -yqq --no-install-recommends libvulkan-dev vulkan-tools
!mkdir -p /usr/share/vulkan/icd.d
!wget -q -P /usr/share/vulkan/icd.d https://raw.githubusercontent.com/haosulab/ManiSkill/main/docker/nvidia_icd.json
!wget -q -O /usr/share/glvnd/egl_vendor.d/10_nvidia.json https://raw.githubusercontent.com/haosulab/ManiSkill/main/docker/10_nvidia.json
!vulkaninfo | head -n 5

!pip install flax jax jaxlib tensorflow absl-py tensorflow-hub tf_agents mediapy
!pip install --upgrade google-auth
!pip install orbax-checkpoint==0.4.4 scipy==1.12.0 keras==2.15.0 tensorflow==2.15.1

!git clone https://github.com/google-deepmind/open_x_embodiment.git
!git clone https://github.com/simpler-env/ManiSkill2_real2sim.git
!pip install -e ./ManiSkill2_real2sim
!git clone https://github.com/simpler-env/SimplerEnv.git
!pip install -e ./SimplerEnv
!mkdir -p ./SimplerEnv/checkpoints

/bin/bash: line 1: nvidia-smi: command not found
Selecting previously unselected package libvulkan1:amd64.
(Reading database ... 128691 files and directories currently installed.)
Preparing to unpack .../libvulkan1_1.3.204.1-2_amd64.deb ...
Unpacking libvulkan1:amd64 (1.3.204.1-2) ...
Selecting previously unselected package libvulkan-dev:amd64.
Preparing to unpack .../libvulkan-dev_1.3.204.1-2_amd64.deb ...
Unpacking libvulkan-dev:amd64 (1.3.204.1-2) ...
Selecting previously unselected package vulkan-tools.
Preparing to unpack .../vulkan-tools_1.3.204.0+dfsg1-1_amd64.deb ...
Unpacking vulkan-tools (1.3.204.0+dfsg1-1) ...
Setting up libvulkan1:amd64 (1.3.204.1-2) ...
Setting up libvulkan-dev:amd64 (1.3.204.1-2) ...
Setting up vulkan-tools (1.3.204.0+dfsg1-1) ...
Processing triggers for libc-bin (2.35-0ubuntu3.8) ...
/sbin/ldconfig.real: /usr/local/lib/libtcm.so.1 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libumf.so.0 is not a symbolic link

/sbin/ldconfig.real: /usr/loc

In [2]:
import site
site.main()

In [3]:
import sys
sys.path.append("./open_x_embodiment/models")

import rt1
import tensorflow_hub as hub
import simpler_env
import json
import numpy as np
from flax.training import checkpoints
import jax
import jax.numpy as jnp
import copy
import tensorflow as tf
from simpler_env.utils.env.observation_utils import get_image_from_maniskill2_obs_dict

2025-04-22 20:16:05.527488: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-22 20:16:05.527563: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-22 20:16:05.530022: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/usr/local/lib/python3.11/dist-packages/gymnasium/envs/registration.py:596: UserWarning: WARN: plugin: shimmy.registration:register_gymnasium_envs raised Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/gymnasium/envs/registration.py", line 594, in load_plugin_envs
    fn()
  File "/usr/local/lib/python3.11/dist-packages/shimmy/r

In [4]:
class RT1Policy:
    def __init__(self, checkpoint_path=None, model=None, variables=None, seqlen=15, rng=None):
        self.model = model
        self._checkpoint_path = checkpoint_path
        self.seqlen = seqlen
        self.rng = rng if rng is not None else jax.random.PRNGKey(0)
        self._run_action_inference_jit = jax.jit(self._run_action_inference)
        if variables:
            self.variables = variables
        else:
            state_dict = checkpoints.restore_checkpoint(checkpoint_path, None)
            self.variables = {
                'params': state_dict['params'],
                'batch_stats': state_dict['batch_stats'],
            }

    def _extract_action_info(self, output_logits):
        time_step_tokens = self.model.num_image_tokens + self.model.num_action_tokens
        output_logits = jnp.reshape(output_logits, (1, self.seqlen, time_step_tokens, -1))
        action_logits = output_logits[:, -1, ...]
        action_logits = action_logits[:, self.model.num_image_tokens - 1 : -1]

        action_logp = jax.nn.softmax(action_logits, axis=-1)
        token_argmax = jnp.argmax(action_logp, axis=-1)
        token_entropy = -jnp.sum(action_logp * jnp.log(action_logp + 1e-8), axis=-1)

        return {
            "token_probs": action_logp,            # (1, num_tokens, vocab_size)
            "token_argmax": token_argmax,          # (1, num_tokens)
            "token_entropy": token_entropy         # (1, num_tokens)
        }

    def _run_action_inference(self, observation, rng):
        act_tokens = jnp.zeros((1, 6, 11))
        batch_obs = jax.tree_util.tree_map(lambda x: jnp.expand_dims(x, 0), observation)
        _, random_rng = jax.random.split(rng)
        output_logits_tuple = self.model.apply(
            self.variables,
            batch_obs,
            act=None,
            act_tokens=act_tokens,
            train=False,
            rngs={'random': random_rng},
        )
        output_logits = output_logits_tuple[0]
        return self._extract_action_info(output_logits)

    def action(self, observation, task_description=None):
        observation = copy.deepcopy(observation)
        if task_description:
            embedding = use_model([task_description])[0].numpy()
            embedding = np.tile(embedding[None, :], (15, 1))
            observation['natural_language_embedding'] = embedding
        image = observation['image']
        image = tf.image.resize(image, (300, 300)).numpy() / 255.0
        observation['image'] = image
        results = []
        self.rng, rng = jax.random.split(self.rng)
        result = self._run_action_inference_jit(observation, rng)
        results.append({
                    "token_probs": jax.device_get(result["token_probs"])[0],
                    "token_argmax": jax.device_get(result["token_argmax"])[0],
                    "token_entropy": jax.device_get(result["token_entropy"])[0],
                })
        return results

In [5]:
!gsutil -m cp -r gs://gdm-robotics-open-x-embodiment/open_x_embodiment_and_rt_x_oss/rt_1_x_jax .

Copying gs://gdm-robotics-open-x-embodiment/open_x_embodiment_and_rt_x_oss/rt_1_x_jax/_METADATA...
Copying gs://gdm-robotics-open-x-embodiment/open_x_embodiment_and_rt_x_oss/rt_1_x_jax/b321733791_75882326_000900000/_METADATA...
Copying gs://gdm-robotics-open-x-embodiment/open_x_embodiment_and_rt_x_oss/rt_1_x_jax/b321733791_75882326_000900000/checkpoint...
==> NOTE: You are downloading one or more large file(s), which would
run significantly faster if you enabled sliced object downloads. This
feature is enabled by default but requires that compiled crcmod be
installed (see "gsutil help crcmod").

Copying gs://gdm-robotics-open-x-embodiment/open_x_embodiment_and_rt_x_oss/rt_1_x_jax/checkpoint...

Operation completed over 4 objects/1.3 GiB.                                      


In [6]:
use_model = hub.load("https://tfhub.dev/google/universal-sentence-encoder-large/5")

In [7]:
rt1x_model = rt1.RT1(
    num_image_tokens=81,
    num_action_tokens=11,
    layer_size=256,
    vocab_size=512,
    use_token_learner=True,
    world_vector_range=(-2.0, 2.0),
)

policy = RT1Policy(
    checkpoint_path='./rt_1_x_jax/',
    model=rt1x_model,
    seqlen=15,
)

In [8]:
from collections import deque
from tqdm import tqdm
import mediapy
import numpy as np

def collect_trajectory(
    env,
    policy,
    task_name,
    task_description,
    steps=10,
    detokenize_actions=True,
    log_frames=True,
    log_probs=True,
    log_entropy=True,
    log_argmax=True,
    return_probs=True
):
    env = simpler_env.make(task_name)
    obs, reset_info = env.reset(seed=np.random.randint(0, 1e6))
    print("Reset info", reset_info)
    frames, trajectory = [], []
    
    image_buffer = deque(maxlen=15)

    for t in range(steps):
        image = get_image_from_maniskill2_obs_dict(env, obs)
        image_buffer.append(image)

        if len(image_buffer) < 15:
            stacked_images = np.stack([image_buffer[0]] * (15 - len(image_buffer)) + list(image_buffer))
        else:
            stacked_images = np.stack(image_buffer)

        obs_dict = {"image": stacked_images.astype(np.uint8)}

        # Get token-level inference
        results = policy.action(obs_dict, task_description)

        timestep_log = {"timestep": t}

        if log_argmax:
            timestep_log["token_argmax"] = [a["token_argmax"].tolist() for a in results]

        if log_entropy:
            timestep_log["token_entropy"] = [a["token_entropy"].tolist() for a in results]

        if log_probs:
            timestep_log["token_probs"] = [
                np.round(a["token_probs"], 5).tolist() for a in results
            ]

        if detokenize_actions:
            token_argmax = [a["token_argmax"] for a in results]
            detokenized_actions = [
                {k: v[0] for k, v in rt1.detokenize_action(
                    t[None, :],
                    policy.model.vocab_size,
                    policy.model.world_vector_range
                ).items()} for t in token_argmax
            ]

            mean_action = {
                k: np.mean([a[k] for a in detokenized_actions], axis=0)
                for k in detokenized_actions[0]
            }

            act_array = np.concatenate([
                mean_action['world_vector'],
                mean_action['rotation_delta'],
                mean_action['gripper_closedness_action']
            ])
            obs, reward, done, truncated, info = env.step(act_array)
            # print(info)
            if log_frames:
                frames.append(image)

            timestep_log["mean_action"] = {k: v.tolist() for k, v in mean_action.items()}
            timestep_log["all_actions"] = [{k: v.tolist() for k, v in a.items()} for a in detokenized_actions]
            timestep_log["info"]=info
            # print(timestep_log['token_argmax'])
            # print(timestep_log["all_actions"])
            # print("truncated",truncated)
            # print("done",done)
            # if done or truncated:
            #     trajectory.append(timestep_log)
            #     break
            if (info['is_grasped'] and info['consecutive_grasp'] and info['lifted_object'] and info['lifted_object_significantly']):
                trajectory.append(timestep_log)
                break

        trajectory.append(timestep_log)

    return frames, trajectory, info['success']

In [9]:
import numpy as np

def convert_numpy(obj):
    if isinstance(obj, dict):
        return {k: convert_numpy(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [convert_numpy(v) for v in obj]
    elif isinstance(obj, np.integer):
        return int(obj)
    elif isinstance(obj, np.floating):
        return float(obj)
    elif isinstance(obj, np.bool_):
        return bool(obj)
    else:
        return obj

def episodeData(episodeNumber,folder):
    import sapien.core as sapien

    sapien.render_config.rt_use_denoiser = False
    
    
    task_name = "google_robot_pick_horizontal_coke_can"
    # task_description = "Pick up the coke can and place it on the table."
    task_description = "Pick up the coke can"
    # task_name = "google_robot_close_drawer"
    # task_description = "Close the drawer"
    # Reset and reinitialize the environment
    if 'env' in locals():
        env.close()
        del env
    env = simpler_env.make(task_name)
    
    # Toggle this for probability logging vs regular trajectory
    RETURN_PROBS = True
    STEPS = 100
    
    frames, trajectory, success = collect_trajectory(
        env,
        policy,
        task_name,
        task_description,
        steps=STEPS,
        return_probs=RETURN_PROBS,
    )
    
    # Save to appropriate file based on mode
    output_file = f"{folder}/json/{success}_{episodeNumber}.json"
    with open(output_file, "w") as f:
        json.dump(convert_numpy(trajectory), f, indent=2)
    mediapy.write_video(f"{folder}/video/{success}_{episodeNumber}.mp4", frames, fps=10)

In [10]:
import warnings

warnings.filterwarnings(
    "ignore",
    category=UserWarning,
    message=r".*env\.robot_uid to get variables from other wrappers is deprecated.*"
)

In [11]:
!rm -rf data
!mkdir data
!mkdir data/json
!mkdir data/video

/usr/lib/python3.11/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [12]:
from tqdm import tqdm

for i in tqdm(range(200), desc="Collecting Episodes"):
    episodeData(i, "data")

[2025-04-22 20:17:09.072] [svulkan2] [warning] Continue without GLFW.
[2025-04-22 20:17:09.073] [svulkan2] [error] Vulkan is incompatible with your driver. You may not use the renderer to render, however, CPU resources will be still available.


In [ ]:
!mkdir data/json_modified

In [ ]:
import os
import json

# Input and output directories
input_dir = "data/json"
output_dir = "data/json_modified"

# Make sure the output directory exists
os.makedirs(output_dir, exist_ok=True)

# Loop through all JSON files
for filename in os.listdir(input_dir):
    if filename.endswith(".json"):
        input_path = os.path.join(input_dir, filename)
        output_path = os.path.join(output_dir, filename)

        with open(input_path, "r") as f:
            data = json.load(f)

        # Remove token_probs from each timestep (list of dicts)
        for timestep in data:
            timestep.pop("token_probs", None)

        # Save to new directory
        with open(output_path, "w") as f:
            json.dump(data, f, indent=2)

In [ ]:
!ls data/json/T* | wc
!ls data/json/F* | wc

In [ ]:
!zip -r data.zip data/